## 필요한 라이브러리

In [1]:
import os
import io
import re
import numpy as np
import pandas as pd
import platform
from PIL import ImageFont, ImageDraw, Image
import matplotlib.pyplot as plt

import cv2
from google.cloud import vision

## Google OCR API 불러오기

In [3]:
# OCR API 불러오기
# \ => /로 변환해야 파일 인식 가능(안바꾸면 유니코드 에러 발생함)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users\Hong Ji Young\Desktop/Health-C/health-c-377913-17227fcd7bd5.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

## 이미지 불러오기

In [4]:
# 이미지 불러오기
path = '../img2.jpg'
with io.open(path, 'rb') as img_file:
    content = img_file.read()

## OCR 인식

In [5]:
image = vision.Image(content=content)
 
response = client.text_detection(image=image)
texts = response.text_annotations

In [24]:
# texts

In [111]:
text = texts[0].description.replace("\n", "")

In [21]:
if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

## Text 전처리

In [154]:
text = re.sub(special_char, '/', text)

In [159]:
text

'//OPEN빼빼로/판매수익금롯데제과/Sweet/Home지역아동센터건립기금으로/사용됩니다/유통기한/밑면표기일까지/연/월/일/식품유형/초콜릿가공품/내포장재질/폴리프로필렌품목보고번호/F//양산/////////////////F//대전///////////////영업장의/명칭/롯데제과//서울시/영등포구/양평로//길///및/소재지원재료명/준초콜릿////설탕//가공유지/싱가포르산//혼합분유/외국산//싱가포르/프랑스/네덜란드/등//전지분유/코코아매스//코코아매스/유당///밀가루/밀/미국산/캐나다산//준초콜릿II/설탕/혼합분유/외국산/싱가포르///프랑스/네덜란드/등//전지분유/코코아매스//가공유지/싱가포르산//코코아매스/유당//시드/곡류가공품//설탕/쇼트닝/가공연유/가공버터/전분가공품/맥아엑기스/전지분골드///정제소금/산료///효소제/효모도조절제/종//기타과당/혼합제제/합성향료/밀가루/비타민C//카제인나트륨/알긴산프로필렌글리콜//혼합제제II/바닐라추출물/합성향밀/대두/우유/땅콩/함유/직사광선/및/습기를/피해/서늘한/곳에/진열//유통/중/변질품은/구입상점/및/본사에서/항상/교환///소비자기본법에/의한/피해/보상//휴지줍는/고운마음//안버리는/밝은마음//부정//불량식품/신고는/국번없이//////달걀/쇠고기/돼지고기/혼입/가능////고객지원센터////////////수신자요금부담http///www/lottecont/co/kr중/초콜릿/표면이/녹아/하얀/반점이/생길/수/있으나//인체에무해하니/드셔도/괜찮습니다/영양정보지방///g////나트륨/////mg/////탄수화물///g/////당류///g////트랜스지방//g포화지방//g////콜레스테롤//mg미만//////일/영양성분/기준치에/대한/비율///은//////kcal/기준이므로/개인의/필요/열량에/따라/다를/수/있습니다///롯데제과단백질//g//눌러서분리배출/해주세요총/내용량////g////kcalSweet/Eco롯데제과와/함께/환경을/지켜요//편리한/분리배출'

In [163]:
gredients_lst = []
special_char = '[^A-Za-z0-9가-힣]'
for first_split in text.split(','):
    material = [re.sub(special_char, ' ', f) for f in first_split.split()]
    for second_split in material:
        for third_split in second_split.split():
            gredients_lst.append(third_split)

In [164]:
print(gredients_lst, end=' ')

['OPEN빼빼로', '판매수익금롯데제과', 'Sweet', 'Home지역아동센터건립기금으로', '사용됩니다', '유통기한', '밑면표기일까지', '연', '월', '일', '식품유형', '초콜릿가공품', '내포장재질', '폴리프로필렌품목보고번호', 'F', '양산', 'F', '대전', '영업장의', '명칭', '롯데제과', '서울시', '영등포구', '양평로', '길', '및', '소재지원재료명', '준초콜릿', '설탕', '가공유지', '싱가포르산', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '코코아매스', '유당', '밀가루', '밀', '미국산', '캐나다산', '준초콜릿II', '설탕', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '가공유지', '싱가포르산', '코코아매스', '유당', '시드', '곡류가공품', '설탕', '쇼트닝', '가공연유', '가공버터', '전분가공품', '맥아엑기스', '전지분골드', '정제소금', '산료', '효소제', '효모도조절제', '종', '기타과당', '혼합제제', '합성향료', '밀가루', '비타민C', '카제인나트륨', '알긴산프로필렌글리콜', '혼합제제II', '바닐라추출물', '합성향밀', '대두', '우유', '땅콩', '함유', '직사광선', '및', '습기를', '피해', '서늘한', '곳에', '진열', '유통', '중', '변질품은', '구입상점', '및', '본사에서', '항상', '교환', '소비자기본법에', '의한', '피해', '보상', '휴지줍는', '고운마음', '안버리는', '밝은마음', '부정', '불량식품', '신고는', '국번없이', '달걀', '쇠고기', '돼지고기', '혼입', '가능', '고객지원센터', '수신자요금부담http', 'www', 'lottecont', 'co', 'kr중', '초콜릿', '표면이', '녹아', '하얀', '반점이',

## 원재료명 list 엑셀 파일 읽어오기
=> excel 파일로 제공되는 파일 가져옴
==> json을 읽어오는 방법도 있음

In [73]:
materials = pd.read_excel('../materials_all.xlsx')

In [74]:
materials.head()

,Unnamed: 0,원재료,대표원재료명,원재료이명
0,0,Abiu열매,Abiu열매,NaN
1,1,Agathi꽃,Agathi꽃,NaN
2,2,Agathi잎,Agathi잎,NaN
3,3,Alder Buckthorn,Alder Buckthorn,NaN
4,4,Algarrobo Blanco,Algarrobo Blanco,NaN


In [165]:
food_materials = pd.DataFrame({'원재료' : list(set(gredients_lst))})
result = pd.merge(materials, food_materials
                   , how="inner", right_on="원재료", left_on="대표원재료명")

In [166]:
list(result["대표원재료명"])

['대두',
 '땅콩',
 '밀',
 '밀가루',
 '연',
 '코코아매스',
 '맥아엑기스',
 '돼지고기',
 '우유',
 '전지분유',
 '은',
 '유당',
 '알긴산프로필렌글리콜',
 '카제인나트륨',
 '설탕',
 '쇼트닝',
 '가공유지',
 '초콜릿',
 '가공버터',
 '전지분골드',
 '준초콜릿',
 '곡류가공품',
 '초콜릿가공품',
 '효소제',
 '혼합제제',
 '나트륨']

In [141]:
lst = []
special_char = ''
for split in text.split():
    x = [re.sub('[^A-Za-z0-9가-힣]', ' ', t) for t in split.split(',')]
    # print(x)
    for j in x:
        lst += j.split()

In [142]:
print(lst, end=' ')

['00OPEN빼빼로', '판매수익금롯데제과', 'Sweet', 'Home지역아동센터건립기금으로', '사용됩니다', '유통기한', '밑면표기일까지', '연', '월', '일', '식품유형', '초콜릿가공품', '내포장재질', '폴리프로필렌품목보고번호', 'F2', '양산', '19780614009498', 'F4', '대전', '19960242067376영업장의', '명칭', '롯데제과', '서울시', '영등포구', '양평로21길', '25및', '소재지원재료명', '준초콜릿', '설탕', '가공유지', '싱가포르산', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '코코아매스', '유당', '밀가루', '밀', '미국산', '캐나다산', '준초콜릿II', '설탕', '혼합분유', '외국산', '싱가포르', '프랑스', '네덜란드', '등', '전지분유', '코코아매스', '가공유지', '싱가포르산', '코코아매스', '유당', '시드', '곡류가공품', '설탕', '쇼트닝', '가공연유', '가공버터', '전분가공품', '맥아엑기스', '전지분골드', '1', '정제소금', '산료', '효소제', '효모도조절제3종', '기타과당', '혼합제제', '합성향료', '밀가루', '비타민C', '카제인나트륨', '알긴산프로필렌글리콜', '혼합제제II', '바닐라추출물', '합성향밀', '대두', '우유', '땅콩', '함유', '직사광선', '및', '습기를', '피해', '서늘한', '곳에', '진열', '유통', '중', '변질품은', '구입상점', '및', '본사에서', '항상', '교환', '소비자기본법에', '의한', '피해', '보상', '휴지줍는', '고운마음', '안버리는', '밝은마음', '부정', '불량식품', '신고는', '국번없이', '1399', '달걀', '쇠고기', '돼지고기', '혼입', '가능', '고객지원센터080', '024', '6060수신자요금부담http', 'www'

In [85]:
text

'00OPEN빼빼로 판매수익금롯데제과 Sweet Home지역아동센터건립기금으로 사용됩니다.유통기한 밑면표기일까지(연.월.일)식품유형 초콜릿가공품|내포장재질 폴리프로필렌품목보고번호 F2(양산) 19780614009498,F4(대전)19960242067376영업장의 명칭 롯데제과㈜ 서울시 영등포구 양평로21길 25및 소재지원재료명 준초콜릿 | (설탕, 가공유지(싱가포르산),혼합분유(외국산·(싱가포르·프랑스·네덜란드 등);전지분유,코코아매스),코코아매스,유당), 밀가루(밀:미국산,캐나다산),준초콜릿II(설탕,혼합분유(외국산(싱가포르 • 프랑스·네덜란드 등);전지분유,코코아매스),가공유지(싱가포르산),코코아매스,유당),시드(곡류가공품),설탕,쇼트닝,가공연유,가공버터,전분가공품,맥아엑기스,전지분골드 1,정제소금,산료), 효소제,효모도조절제3종, 기타과당,혼합제제(합성향료,밀가루,비타민C, 카제인나트륨,알긴산프로필렌글리콜),혼합제제II(바닐라추출물,합성향밀,대두,우유,땅콩 함유•직사광선 및 습기를 피해 서늘한 곳에 진열, 유통 중 변질품은 구입상점 및 본사에서 항상 교환 • 소비자기본법에 의한 피해 보상• 휴지줍는 고운마음, 안버리는 밝은마음• 부정, 불량식품 신고는 국번없이 1399•달걀,쇠고기,돼지고기 혼입 가능•●• 고객지원센터080-024-6060수신자요금부담http://www.lottecont.co.kr중 초콜릿 표면이 녹아 하얀 반점이 생길 수 있으나, 인체에무해하니 드셔도 괜찮습니다.영양정보지방 11g 20%나트륨 140 mg7% | 탄수화물 24g7% | 당류 13g 13%트랜스지방 0g포화지방 6g 40%콜레스테롤 5mg미만 1 %*1일 영양성분 기준치에 대한 비율(%)은 2,000kcal 기준이므로 개인의 필요 열량에 따라 다를 수 있습니다.1 롯데제과단백질 3g5%눌러서분리배출 해주세요총 내용량 39 g205 kcalSweet Eco롯데제과와 함께 환경을 지켜요△ 편리한 분리배출'

In [99]:
lst = [i for i in '준초콜릿, 설탕, 가공유지, 혼합분유, 전지분유, 코코아매스, 유당, 밀가루, 밀, 준초콜릿II, 시드, 곡류가공품, 쇼트닝, 가공연유, 가공버터, 전분가공품, 맥아엑기스, 전지분골드I, 정제소금, 산도조절제3종, 기타과당, 혼합제제I, 합성향료, 비타민C,  카제인나트륨, 알긴산프로필렌글리콜, 혼합제제II, 바닐라추출물, 효소제, 효모, 대두, 우유, 땅콩, 달걀, 쇠고기, 돼지고기'.split(', ')]

In [100]:
len(set(lst))

36